In [1]:
import paramiko
import datetime
import pandas as pd
import numpy as np
import os

print(os.getcwd())
print(datetime.datetime.now())

/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/all_LR_returned
2020-05-15 11:05:53.857439


# 107

In [6]:
host_107 = "107.191.32.220" #hard-coded
port_107 = 22
transport_107 = paramiko.Transport((host_107, port_107))

password_107 = "juba-jl1" #hard-coded
username_107 = "jian" #hard-coded
transport_107.connect(username = username_107, password = password_107)
sftp_107 = paramiko.SFTPClient.from_transport(transport_107)

In [17]:
folder_remote_107_others="/mnt/drv5/lr_biglots_data/download_logs/others/"
list_mappings=sftp_107.listdir(folder_remote_107_others)
list_mappings=[x for x in list_mappings if "mapping" in x]
df_remote_mapping=pd.DataFrame({"remote_base_name":list_mappings},index=[0]*len(list_mappings))
df_remote_mapping['Quarter']=df_remote_mapping['remote_base_name'].apply(lambda x: x.split("_mapping_file_")[1][:6])
df_remote_mapping=df_remote_mapping.sort_values("remote_base_name",ascending=False)
df_remote_mapping=df_remote_mapping.drop_duplicates("Quarter")

In [18]:
for file in df_remote_mapping['remote_base_name'].tolist():
    sftp_107.get(folder_remote_107_others+file,"./"+file)
sftp_107.close()
transport_107.close()


# 66

In [2]:
host_66 = "66.55.157.58" #hard-coded
port_66 = 22
transport_66 = paramiko.Transport((host_66, port_66))

password_66 = "jubaplus2019" #hard-coded
username_66 = "jian" #hard-coded
transport_66.connect(username = username_66, password = password_66)
sftp_66 = paramiko.SFTPClient.from_transport(transport_66)

In [3]:
folder_remote_66_others="/data/jian/BL_LR_download/others/"
list_mappings=sftp_66.listdir(folder_remote_66_others)
list_mappings=[x for x in list_mappings if "mapping" in x]
df_remote_mapping=pd.DataFrame({"remote_base_name":list_mappings},index=[0]*len(list_mappings))
df_remote_mapping['Quarter']=df_remote_mapping['remote_base_name'].apply(lambda x: x.split("_mapping_file_")[1][:6])
df_remote_mapping['remote_mt_time']=df_remote_mapping['remote_base_name'].apply(lambda x: sftp_66.stat(folder_remote_66_others+x).st_mtime)
df_remote_mapping=df_remote_mapping.sort_values(["remote_base_name","remote_mt_time"],ascending=[False,False])
df_remote_mapping=df_remote_mapping.drop_duplicates("Quarter")
df_remote_mapping

,remote_base_name,Quarter,remote_mt_time
0,CL_BigLots_BL_mapping_file_2020Q1_JL_2020-05-1...,2020Q1,1589543111


In [4]:
df_remote_mapping['remote_base_name'].tolist()

['CL_BigLots_BL_mapping_file_2020Q1_JL_2020-05-13_20200514_172449_0000.psv.gz']

In [5]:
quarters_already_copied=os.listdir("./")
quarters_already_copied=[x for x in quarters_already_copied if "_mapping_file_" in x]
quarters_already_copied=[x.split("_mapping_file_")[1][:6] for x in quarters_already_copied]
quarters_already_copied=set(quarters_already_copied)
quarters_already_copied

{'2018Q4', '2019Q1', '2019Q2', '2019Q3', '2019Q4'}

In [6]:
df_remote_mapping=df_remote_mapping[~df_remote_mapping['Quarter'].isin(quarters_already_copied)]
df_remote_mapping

,remote_base_name,Quarter,remote_mt_time
0,CL_BigLots_BL_mapping_file_2020Q1_JL_2020-05-1...,2020Q1,1589543111


In [7]:
df_remote_mapping=df_remote_mapping.drop_duplicates("Quarter")

In [8]:
for file in df_remote_mapping['remote_base_name'].tolist():
    sftp_66.get(folder_remote_66_others+file,"./"+file)
sftp_66.close()
transport_66.close()

# QC: 20Q1 vs 19Q4

In [10]:
df_2020Q1_mapping=pd.read_csv("./CL_BigLots_BL_mapping_file_2020Q1_JL_2020-05-13_20200514_172449_0000.psv.gz",dtype=str,
                             sep="|",compression="gzip")

df_2019Q4_mapping=pd.read_csv("./CL_BigLots_BL_mapping_file_2019Q4_JL_2020-02-14_20200215_020719_0000.psv.gz",dtype=str,
                             sep="|",compression="gzip")

In [11]:
df_2020Q1_mapping=df_2020Q1_mapping.rename(columns={"Customer_Link":"Customer_Link_Q1","segment":"segment_Q1"})
df_2019Q4_mapping=df_2019Q4_mapping.rename(columns={"Customer_Link":"Customer_Link_Q4","segment":"segment_Q4"})

In [12]:
list_Q1_seg=df_2020Q1_mapping['segment_Q1'].unique().tolist()
list_Q1_seg.sort()

In [13]:
df_2020Q1_mapping.head(2)

,Customer_Link_Q1,customer_id_hashed,segment_Q1
0,XY1468JYDucT7nAHqbP0nTwx2g2YXFd7oE-Ns_ksNkqyDyJrw,fb06e28fae22be4687038ceb72eed543b7d7af4a6cbbfc...,T_Legacy_Lapsed1318_L_P_Above50K_SingleFamily_...
1,Xi1468LHmoTx2mIkTbbURLh3KFxI3Hmvm6WQhQTQlSj8ha...,65abd0f8c70be4af489bee446cf0901059f5ab8338b490...,T_Legacy_Active_H_P_Above50K_SingleFamily_Comp...


In [14]:
df_both=pd.merge(df_2020Q1_mapping,df_2019Q4_mapping,on="customer_id_hashed",how="outer")

In [15]:
df_both.shape

(27389002, 5)

In [16]:
df_both[df_both['Customer_Link_Q1']==df_both['Customer_Link_Q4']].shape

(17320522, 5)

In [17]:
df_both=df_both[['customer_id_hashed','Customer_Link_Q1','Customer_Link_Q4','segment_Q1','segment_Q4']].sort_values("customer_id_hashed")

In [18]:
df_local_uploaded=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2020_Q1/BL_mapping_file_2020Q1_JL_2020-05-13.csv",
                             dtype=str)

In [19]:
df_local_uploaded=df_local_uploaded.rename(columns={"segment":"segment_Q1_before_upload"})

In [20]:
df_2020Q1_mapping.head(2)

,Customer_Link_Q1,customer_id_hashed,segment_Q1
0,XY1468JYDucT7nAHqbP0nTwx2g2YXFd7oE-Ns_ksNkqyDyJrw,fb06e28fae22be4687038ceb72eed543b7d7af4a6cbbfc...,T_Legacy_Lapsed1318_L_P_Above50K_SingleFamily_...
1,Xi1468LHmoTx2mIkTbbURLh3KFxI3Hmvm6WQhQTQlSj8ha...,65abd0f8c70be4af489bee446cf0901059f5ab8338b490...,T_Legacy_Active_H_P_Above50K_SingleFamily_Comp...


In [21]:
df_2020Q1_mapping.shape,df_local_uploaded.shape

((20003623, 3), (20003623, 3))

In [22]:
df_local_uploaded.head(2)

,customer_id_hashed,email_address_hash,segment_Q1_before_upload
0,f68ce02330d03f94b2e93975fb3bd285f3d5a13dba52dc...,31273951dfe437775d15dde53e914b75f16d83f21f9433...,T_Legacy_Active_H_P_Above50K_MultiFamilyOrCond...
1,52654f3c1b0eebc123084740e81724fd5c355a1bf921c7...,c2ab5925b536dd4dbf66bf1a82393dd32ab11f5a1ee171...,T_Legacy_Active_H_P_Above50K_MultiFamilyOrCond...


In [23]:
df=pd.merge(df_local_uploaded,df_2020Q1_mapping,on="customer_id_hashed",how="inner")
df.shape

(20003623, 5)